In [0]:
import pandas as pd
import numpy as np 
from sklearn.utils import shuffle
import cv2
import glob
import random
from PIL import Image
emotions = ['boredm','confusion','engagement','frustration'] 
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

In [0]:
def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("../dataset/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction  

In [0]:
def get_img(ems):
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  pixel=[]
  usage=[]
  em=[]
  for xx in range(len(ems)):
    tr,te=get_files(ems[xx])
    for item in tr:
      image = cv2.imread(item) #open image
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
      gray=np.asarray(Image.fromarray(gray).resize((48,48)))
      clahe_image = clahe.apply(gray)
      clahe_image=clahe_image.reshape(48*48)
      s=""
      for i in clahe_image:
        s=s+str(i)+" "
      em.append(xx)
      pixel.append(s)
      usage.append('train')
    for item in te:
      image = cv2.imread(item) #open image
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
      gray=np.asarray(Image.fromarray(gray).resize((48,48)))
      clahe_image = clahe.apply(gray)
      clahe_image=clahe_image.reshape(48*48)
      s=""
      for i in clahe_image:
        s=s+str(i)+" "
      em.append(xx)
      pixel.append(s)
      usage.append('test')
    print('done',ems[xx])
  df={'emotion':pd.Series(em),'pixels':pd.Series(pixel),'usage':pd.Series(usage)}
  return df

In [0]:
d=get_img(emotions)

done boredm
done confusion
done engagement
done frustration


In [0]:
df=pd.DataFrame(d)
df

,emotion,pixels,usage
0,0,255 234 212 177 163 170 174 135 177 234 176 89...,train
1,0,71 50 28 42 86 89 88 88 83 71 62 59 64 62 66 6...,train
2,0,14 57 99 156 149 137 120 106 109 106 126 121 1...,train
3,0,227 198 128 71 68 47 42 28 68 106 158 175 166 ...,train
4,0,248 220 205 142 78 40 42 62 88 113 137 158 152...,train
...,...,...,...
3802,3,234 234 198 92 121 163 110 59 60 57 50 28 100 ...,test
3803,3,92 28 42 142 178 173 149 144 152 184 184 177 1...,test
3804,3,42 28 28 78 102 130 156 151 133 113 99 108 99 ...,test
3805,3,64 35 50 50 44 38 32 26 48 78 99 120 117 111 1...,test


In [0]:
df.to_pickle("../res/Images_data.pkl")